In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os
import glob

In [2]:
ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

In [3]:
#Note 1/18/23: Flux tower footprint (fluxproper) OR expanded flux area (fluxdiss) OR all gce (gce_clip)

# gce_clip = 'F:/Wetlands shapefiles/GA_shapefile_wetlands/GA_sapelodissolve.shp'

# gce_fluxdiss = 'F:/Wetlands shapefiles/GA_shapefile_wetlands/GA_fluxdissolve.shp'

# gce_fluxproper = 'F:/Wetlands shapefiles/GCE flux tower/2014-climatology.shp'

gce_expandedflux = 'F:/Wetlands shapefiles/GCE flux tower/expanded_flux.shp' ## Added 9/18/23 - fix shapefile problems!

flux_ga = geemap.shp_to_ee(gce_expandedflux)

## DEM
elevation = ee.Image('USGS/3DEP/10m').clip(flux_ga) ##This is 1/3 arc second, or 10 m.

In [5]:
def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

#MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

In [6]:
# Function to add the date band
def add_date_band(image):
    # Get the acquisition date
    date = ee.Date(image.get('system:time_start'))
    
    # Convert the date to milliseconds since Unix epoch
    date_millis = date.millis()
    
    # Create an image with a single band representing the acquisition date
    date_image = ee.Image.constant(date_millis).int64().rename('acquisition_date')
    
    # Add the date image as a band to the original image
    return image.addBands(date_image)


In [7]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

In [8]:
def addevi(image):
    evi = ee.Image(0).expression(
        '(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)', {
            'RED': image.select('SR_B4'),
            'NIR': image.select('SR_B5'),
            'BLUE': image.select('SR_B2'),

        })
    
    return image.addBands(evi.rename('evi'))

In [10]:
#Loading gee datasets
l8_col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(2013, 2023,'year')) \
    .filterBounds(flux_ga) \
    .map(maskL5sr).map(addFLATSL8).map(add_date_band)

In [13]:
# Display the percentage image on the map
# Map = geemap.Map()
# Map.centerObject(flux_ga, 8)
# Map.addLayer(l8_col.first())
# Map.addLayer(flux_ga)
# Map

In [15]:
## Extracting sentinel pixel values
collection = l8_col

df = pd.DataFrame(collection.aggregate_array('system:id').getInfo(),columns = ['Image'], index = None)

#print(df)

calibr = []
for i in range(len(df)):
    image = ee.Image(addevi(maskFLATS(addFLATSL8(maskL5sr(add_date_band(ee.Image(df.loc[i, 'Image'])))))))
    try:
        region = flux_ga
        samplex = geemap.ee_to_pandas(image.addBands(image.pixelLonLat()).sampleRegions(collection = region, scale = 30, geometries = True))
        calibr.append(samplex) 
    except Exception:
        continue #Unless this exception is included, images with no pixels in shapefile (due to clouds) will result in error

image_list = pd.concat(calibr)

print(image_list)

       Shape_Leng ATTRIBUTE    Shape_Area    Shape_Le_1  \
0    14200.477256    E2EM1N  1.761018e+06  14200.477256   
1    14200.477256    E2EM1N  1.761018e+06  14200.477256   
2    14200.477256    E2EM1N  1.761018e+06  14200.477256   
3    14200.477256    E2EM1N  1.761018e+06  14200.477256   
4    14200.477256    E2EM1N  1.761018e+06  14200.477256   
..            ...       ...           ...           ...   
986  14200.477256    E2EM1N  1.761018e+06  14200.477256   
987  14200.477256    E2EM1N  1.761018e+06  14200.477256   
988  14200.477256    E2EM1N  1.761018e+06  14200.477256   
989  14200.477256    E2EM1N  1.761018e+06  14200.477256   
990  14200.477256    E2EM1N  1.761018e+06  14200.477256   

                       WETLAND_TY       ACRES  ST_EMSD  ST_URAD   latitude  \
0    Estuarine and Marine Wetland  435.156601        0      576  31.459578   
1    Estuarine and Marine Wetland  435.156601        0      576  31.459307   
2    Estuarine and Marine Wetland  435.156601        0   

In [16]:
image_list['Date'] = pd.to_datetime(image_list['acquisition_date'], unit='ms')
image_list['Date']

0     2013-03-28 15:56:07.945
1     2013-03-28 15:56:07.945
2     2013-03-28 15:56:07.945
3     2013-03-28 15:56:07.945
4     2013-03-28 15:56:07.945
                ...          
986   2023-09-06 16:00:45.517
987   2023-09-06 16:00:45.517
988   2023-09-06 16:00:45.517
989   2023-09-06 16:00:45.517
990   2023-09-06 16:00:45.517
Name: Date, Length: 420763, dtype: datetime64[ns]

In [17]:
##Export as csv
in_fc = image_list
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'flux_evi.csv')
# image_list.to_csv(out_csv, index = False)